In [1]:
#imports libraries that will be utilized
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import requests
import scipy.stats as sts
import scipy as spy
import scipy.integrate as integrate
import scipy.optimize as opt
import random
import numpy.linalg as lin

Question 1:

Literal a)

In [2]:
#loads information, names columns, and specifies initial parameter values and number of simulations.
dt1=pd.read_csv('NewMacroSeries.txt',sep=",",header=None)
dt1.columns=["c","k","w","r","y"]
beta=0.99
alpha_0=1/2
rho_0=1/2
mu_0=9
sigma_0=1/2
sim=1000

#Initial weighting matrix
W=np.eye(6)

#Function 7: Calculates k(t+1) based on alpha, beta, z(t) and k(t)
def function7k1(al,be,zt,kt):
    kt1=al*be*np.exp(zt)*(kt**al)
    return kt1

#Initial k value ("k1")
k1=np.mean(dt1['k'])

#Generates matrix with random cumulative probability values
np.random.seed(25)  # Set the random number seed so it gives same answers every time
S_0 = sts.uniform.rvs(0, 1, size=(len(dt1['c']),sim))

#Function that calculates error based on a cumulative probability value
def epsilonma(matrix,sigma):
    epsm=sts.norm.ppf(matrix,loc=0,scale=sigma)
    return epsm

#Function that calculates a simulated z's based on errors, rho, mu and an initial z
def simz(errors,rho,mu,z0):
    simulatedz=np.array(errors)
    simulatedz[0,:]=np.array(rho*z0+(1-rho)*mu+errors[0,:])
    
    for l in range (1,len(dt1['c'])):
        simulatedz[l,:]=np.array(rho*simulatedz[l-1,:]+(1-rho)*mu+errors[l,:])
    
    return simulatedz

#Function that calculates a simulated k(t) based on z(t), alpha, beta and k(t)
#(Considers an initial k1 value)
def simk(simz, alpha, be, k_1):
    simulatedk=np.array(simz)
    simulatedk[0,:]=k_1
    
    for row in range (1, len(dt1['c'])):
        simulatedk[row,:]=function7k1(alpha,be,simz[row-1,:],simulatedk[row-1,:])
                                
    
    return simulatedk

#Function that calculates a simulated k(t+1) based on z(t), alpha, beta and k(t)
#(Considers an initial k1 value)
def simk1(simz, alpha, be, k_1):
    simulatedk=np.array(simz)
    simulatedk[0,:]=function7k1(alpha, be, simz[1,:],k_1)
    
    for row in range (1, len(dt1['c'])):
        simulatedk[row,:]=function7k1(alpha,be,simz[row,:],simulatedk[row-1,:])
                                
    
    return simulatedk

#Calculates data moments
m1=np.mean(dt1['c'])
m2=np.mean(dt1['k'])
m3=np.mean(dt1['c'])/np.mean(dt1['k'])
m4=np.var(dt1['y'])

modct=np.zeros((len(dt1['c'])-1))
modct1=np.zeros((len(dt1['c'])-1))

for count in range (0, len(dt1['c'])-1):
    modct[count]=dt1['c'][count]
    modct1[count]=dt1['c'][count+1]

corrct, pvalct=sts.pearsonr(modct,modct1)
m5=corrct

corrck, pvalck = sts.pearsonr(dt1['c'],dt1['k'])
m6=corrck

#Defines optimization criterion
def crit(params):
    alpha,rho,mu,sigma = params

    #Creates objects to store simulated moments
    sm1=[]
    sm2=[]
    sm3=[]
    sm4=[]
    sm5=[]
    sm6=[]
    modcst=np.zeros((len(dt1['c'])-1,sim))
    modcst1=np.zeros((len(dt1['c'])-1,sim))
    modct=np.zeros((len(dt1['c'])-1))
    modct1=np.zeros((len(dt1['c'])-1))
    
    
    z0=mu
    
    #Calculates variables as per problem set's equations
    errors=np.array(epsilonma(S_0, sigma))
    zst=np.array(simz(errors, rho,mu,z0))
    kst=np.array(simk(zst,alpha,beta,k1))
    kst1=np.array(simk1(zst,alpha,beta,k1))
    wst=np.array((1-alpha)*np.exp(zst)*(kst**alpha))
    rst=np.array(alpha*np.exp(zst)*(kst**(alpha-1)))
    cst=np.array((rst*kst)+wst-kst1)
    yst=np.array(np.exp(zst)*(kst**alpha))
    
    #Calculates simulated moments
    for i in range (0, sim):
    
        sm1.append(np.mean(cst[:,i]))
        sm2.append(np.mean(kst[:,i]))
        sm3.append(np.mean(cst[:,i]/kst[:,i]))
        sm4.append(np.var(yst[:,i]))
        
        for count in range (0, len(dt1['c'])-1):
            modcst[count,i]=cst[count,i]
            modcst1[count,i]=cst[count+1,i]
        
        corrm5, pvalm5 = sts.pearsonr(modcst[:,i],modcst1[:,i])
        sm5.append(corrm5)
        
        corrm6, pvalm6 = sts.pearsonr(cst[:,i],kst[:,i])
        sm6.append(corrm6)
        
    #Calculates errors (Difference between simulated moments and data moments)
    err1=(np.mean(sm1)-m1)/m1
    err2=(np.mean(sm2)-m2)/m2
    err3=(np.mean(sm3)-m3)/m3
    err4=(np.mean(sm4)-m4)/m4
    err5=(np.mean(sm5)-m5)/m5
    err6=(np.mean(sm6)-m6)/m6

    #Creates array of errors
    err=np.array([err1, err2, err3, err4, err5, err6])
    
    #Calculates estimation criterion
    estimator=err.T @ W @ err
    
    return estimator

#Performs optimization starting with initial parameters
params_0=np.array([alpha_0,rho_0,mu_0, sigma_0])
results_uncstr = opt.minimize(crit, params_0, method='L-BFGS-B',bounds=((0.01,0.99), (-0.99, 0.99),(5,14),(0.01, 1.1)),tol=1e-15)
alpha, rho, mu, sigma = results_uncstr.x

#Prints estimated parameters
print("Estimated Alpha = ", alpha)
print("Estimated Rho = ", rho)
print("Estimated Mu = ", mu)
print("Estimated Sigma = ", sigma)

#Calculates final values of moments with optimized parameters (repeats the process described in
#   "optimization criteria")
sm1=[]
sm2=[]
sm3=[]
sm4=[]
sm5=[]
sm6=[]
modcst=np.zeros((len(dt1['c'])-1,sim))
modcst1=np.zeros((len(dt1['c'])-1,sim))
modct=np.zeros((len(dt1['c'])-1))
modct1=np.zeros((len(dt1['c'])-1))
    
    
z0=mu
    
errors=np.array(epsilonma(S_0, sigma))
zst=np.array(simz(errors,rho,mu,z0))
kst=np.array(simk(zst,alpha,beta,k1))
kst1=np.array(simk1(zst,alpha,beta,k1))
wst=np.array((1-alpha)*np.exp(zst)*(kst**alpha))
rst=np.array(alpha*np.exp(zst)*(kst**(alpha-1)))
cst=np.array((rst*kst)+wst-kst1)
yst=np.array(np.exp(zst)*(kst**alpha))
    
for i in range (0, sim):
    
    sm1.append(np.mean(cst[:,i]))
    sm2.append(np.mean(kst[:,i]))
    sm3.append(np.mean(cst[:,i]/kst[:,i]))
    sm4.append(np.var(yst[:,i]))
        
    for count in range (0, len(dt1['c'])-1):
        modcst[count,i]=cst[count,i]
        modcst1[count,i]=cst[count+1,i]
        
    corrm5, pvalm5 = sts.pearsonr(modcst[:,i],modcst1[:,i])
    sm5.append(corrm5)
        
    corrm6, pvalm6 = sts.pearsonr(cst[:,i],kst[:,i])
    sm6.append(corrm6)


err1=(np.mean(sm1)-m1)/m1
err2=(np.mean(sm2)-m2)/m2
err3=(np.mean(sm3)-m3)/m3
err4=(np.mean(sm4)-m4)/m4
err5=(np.mean(sm5)-m5)/m5
err6=(np.mean(sm6)-m6)/m6

err=np.array([err1, err2, err3, err4, err5, err6])

#Prints final vector of moment differences
print("Vector of differences = ", err)

#Prints value of final estimator
print("Criterion function value =",crit(results_uncstr.x))

#Calculates covariance matrix
nmom=6
Ematrix=np.zeros((nmom,sim))
DataMom=np.array([m1,m2,m3,m4,m5,m6])
SimMom=np.array([sm1,sm2,sm3,sm4,sm5,sm6])

for col in range (0,sim):
    for row in range (0,nmom):
        Ematrix[row,col]=(SimMom[row,col]-DataMom[row])/DataMom[row]
        
sig2=(1/sim)*(Ematrix @ Ematrix.T)

StandardErra=[]

#Calculates and prints Standard errors. Order: m1, m2, m3, m4, m5, m6
for row in range (0,nmom):
    StandardErra.append((sig2[row,row])**1/2)

print("Standard Errors of Moments = ", StandardErra)

Estimated Alpha =  0.42204300835825365
Estimated Rho =  0.9182326252836372
Estimated Mu =  9.910569270292159
Estimated Sigma =  0.08864077605595208
Vector of differences =  [-1.34117872e-03  1.34696106e-03  1.35027716e-03 -4.65933474e-06
  6.96054636e-05  6.68699414e-05]
Criterion function value = 5.445651085003343e-06
Standard Errors of Moments =  [0.016298636112670507, 0.016211112998062462, 8.454077168763666e-06, 0.33285564430799597, 0.000341647818098008, 0.00033807597626842343]


Literal b):

***Run after running literal a)

In [3]:
#Performs the same calculation showed in a, but includes the step-2 weighting matrix

#Calculates 2-step weighting matrix
W2=lin.inv(sig2)

def critb(params):
    alpha,rho,mu,sigma = params

    sm1=[]
    sm2=[]
    sm3=[]
    sm4=[]
    sm5=[]
    sm6=[]
    modcst=np.zeros((len(dt1['c'])-1,sim))
    modcst1=np.zeros((len(dt1['c'])-1,sim))
    modct=np.zeros((len(dt1['c'])-1))
    modct1=np.zeros((len(dt1['c'])-1))
    
    
    z0=mu
    
    errors=np.array(epsilonma(S_0, sigma))
    zst=np.array(simz(errors, rho,mu,z0))
    kst=np.array(simk(zst,alpha,beta,k1))
    kst1=np.array(simk1(zst,alpha,beta,k1))
    wst=np.array((1-alpha)*np.exp(zst)*(kst**alpha))
    rst=np.array(alpha*np.exp(zst)*(kst**(alpha-1)))
    cst=np.array((rst*kst)+wst-kst1)
    yst=np.array(np.exp(zst)*(kst**alpha))
    
    
    for i in range (0, sim):
    
        sm1.append(np.mean(cst[:,i]))
        sm2.append(np.mean(kst[:,i]))
        sm3.append(np.mean(cst[:,i]/kst[:,i]))
        sm4.append(np.var(yst[:,i]))
        
        for count in range (0, len(dt1['c'])-1):
            modcst[count,i]=cst[count,i]
            modcst1[count,i]=cst[count+1,i]
        
        corrm5, pvalm5 = sts.pearsonr(modcst[:,i],modcst1[:,i])
        sm5.append(corrm5)
        
        corrm6, pvalm6 = sts.pearsonr(cst[:,i],kst[:,i])
        sm6.append(corrm6)
        
    
    err1=(np.mean(sm1)-m1)/m1
    err2=(np.mean(sm2)-m2)/m2
    err3=(np.mean(sm3)-m3)/m3
    err4=(np.mean(sm4)-m4)/m4
    err5=(np.mean(sm5)-m5)/m5
    err6=(np.mean(sm6)-m6)/m6

    
    err=np.array([err1, err2, err3, err4, err5, err6])
    
    estimatorb=err.T @ W2 @ err
    
    return estimatorb

results_uncstrb = opt.minimize(critb, params_0, method='L-BFGS-B',bounds=((0.01,0.99), (-0.99, 0.99),(5,14),(0.01, 1.1)),tol=1e-15)
alpha_b, rho_b, mu_b, sigma_b = results_uncstrb.x
print("Estimated Alpha = ", alpha_b)
print("Estimated Rho = ", rho_b)
print("Estimated Mu = ", mu_b)
print("Estimated Sigma = ", sigma_b)

sm1_b=[]
sm2_b=[]
sm3_b=[]
sm4_b=[]
sm5_b=[]
sm6_b=[]
modcst_b=np.zeros((len(dt1['c'])-1,sim))
modcst1_b=np.zeros((len(dt1['c'])-1,sim))
modct_b=np.zeros((len(dt1['c'])-1))
modct1_b=np.zeros((len(dt1['c'])-1))
    
    
z0_b=mu_b
    
errors_b=np.array(epsilonma(S_0, sigma_b))
zst_b=np.array(simz(errors_b,rho_b,mu_b,z0_b))
kst_b=np.array(simk(zst_b,alpha_b,beta,k1))
kst1_b=np.array(simk1(zst_b,alpha_b,beta,k1))
wst_b=np.array((1-alpha_b)*np.exp(zst_b)*(kst_b**alpha_b))
rst_b=np.array(alpha_b*np.exp(zst_b)*(kst_b**(alpha_b-1)))
cst_b=np.array((rst_b*kst_b)+wst_b-kst1_b)
yst_b=np.array(np.exp(zst_b)*(kst_b**alpha_b))
    
for i in range (0, sim):
    
    sm1_b.append(np.mean(cst_b[:,i]))
    sm2_b.append(np.mean(kst_b[:,i]))
    sm3_b.append(np.mean(cst_b[:,i]/kst_b[:,i]))
    sm4_b.append(np.var(yst_b[:,i]))
        
    for count in range (0, len(dt1['c'])-1):
        modcst_b[count,i]=cst_b[count,i]
        modcst1_b[count,i]=cst_b[count+1,i]
        
    corrm5b, pvalm5b = sts.pearsonr(modcst_b[:,i],modcst1_b[:,i])
    sm5_b.append(corrm5b)
        
    corrm6b, pvalm6b = sts.pearsonr(cst_b[:,i],kst_b[:,i])
    sm6_b.append(corrm6b)

err1b=(np.mean(sm1_b)-m1)/m1
err2b=(np.mean(sm2_b)-m2)/m2
err3b=(np.mean(sm3_b)-m3)/m3
err4b=(np.mean(sm4_b)-m4)/m4
err5b=(np.mean(sm5_b)-m5)/m5
err6b=(np.mean(sm6_b)-m6)/m6

errb=np.array([err1b, err2b, err3b, err4b, err5b, err6b])

print("Vector of differences = ", errb)

print("Criterion function value =",critb(results_uncstrb.x))

Ematrixb=np.zeros((nmom,sim))
SimMomb=np.array([sm1_b,sm2_b,sm3_b,sm4_b,sm5_b,sm6_b])

for col in range (0,sim):
    for row in range (0,nmom):
        Ematrixb[row,col]=(SimMomb[row,col]-DataMom[row])/DataMom[row]
        
sig2b=(1/sim)*(Ematrixb @ Ematrixb.T)

StandardErrb=[]

for row in range (0,nmom):
    StandardErrb.append((sig2b[row,row])**1/2)

print("Standard Errors of Moments = ", StandardErrb)

Estimated Alpha =  0.42193439798329185
Estimated Rho =  0.934302710395832
Estimated Mu =  9.909510636280412
Estimated Sigma =  0.08142620018876591
Vector of differences =  [-0.00265372 -0.00043168  0.00105167 -0.0067238   0.00746498  0.00747556]
Criterion function value = 0.7629278376345539
Standard Errors of Moments =  [0.020262385537211478, 0.02011038544928693, 8.336497074395062e-06, 0.4122785238936569, 0.00033249389457880427, 0.0003279332136009513]


Checking convergence of optimization routines:

In [9]:
print(results_uncstr)
print("")
print(results_uncstrb)

      fun: 5.445651085003343e-06
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-4.18434276e-10, -1.22311558e-10, -5.32783724e-11, -1.99899776e-11])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 805
      nit: 97
   status: 0
  success: True
        x: array([0.42204301, 0.91823263, 9.91056927, 0.08864078])

      fun: 0.7629278376345539
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 2.42583731e-04, -1.74971149e-05,  2.59792188e-06,  1.60649272e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 470
      nit: 39
   status: 0
  success: True
        x: array([0.4219344 , 0.93430271, 9.90951064, 0.0814262 ])
